# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [1]:
import numpy as np
from radam import radam

print('Loading data ...')

data_root='timit_11/timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [2]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [3]:
VAL_RATIO = 0.05

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1168435, 429)
Size of validation set: (61497, 429)


In [4]:
# mean = train_x.mean(axis=0)
# std = train_x.std(axis=0)
# train_x = (train_x - mean) / std
# val_x = (val_x - mean) / std
# test = (test - mean) / std

Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [5]:
BATCH_SIZE = 1024

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

<ipython-input-2-f6af567917fe>:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = y.astype(np.int)


Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [6]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

80

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [7]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 2048)
        self.layer2 = nn.Linear(2048, 2048)
        self.layer3 = nn.Linear(2048, 2048)
        self.layer4 = nn.Linear(2048, 2048)
        self.layer5 = nn.Linear(2048, 2048)
        self.layer6 = nn.Linear(2048, 2048)
        self.layer7 = nn.Linear(2048, 2048)
        self.layer8 = nn.Linear(2048, 2048)
        self.out = nn.Linear(2048, 39)
        torch.nn.init.xavier_uniform_(self.layer1.weight)
        torch.nn.init.xavier_uniform_(self.layer2.weight)
        torch.nn.init.xavier_uniform_(self.layer3.weight)
        torch.nn.init.xavier_uniform_(self.layer4.weight)
        torch.nn.init.xavier_uniform_(self.layer5.weight)
        torch.nn.init.xavier_uniform_(self.layer6.weight)
        torch.nn.init.xavier_uniform_(self.layer7.weight)
        torch.nn.init.xavier_uniform_(self.layer8.weight)
        torch.nn.init.xavier_uniform_(self.out.weight)
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.6)
        self.bn1 = nn.BatchNorm1d(2048)
        self.bn2 = nn.BatchNorm1d(2048)
        self.bn3 = nn.BatchNorm1d(2048)
        self.bn4 = nn.BatchNorm1d(2048)
        self.bn5 = nn.BatchNorm1d(2048)
        self.bn6 = nn.BatchNorm1d(2048)
        self.bn7 = nn.BatchNorm1d(2048)
        self.bn8 = nn.BatchNorm1d(2048)
        self.act_fn = nn.ReLU()

    def forward(self, x):
        
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.act_fn(x)
        x = self.dropout2(x)
        
        x = self.layer2(x)
        x = self.bn2(x)
        x = self.act_fn(x)
        x = self.dropout2(x)
        
        x = self.layer3(x)
        x = self.bn3(x)
        x = self.act_fn(x)
        x = self.dropout2(x)
        
        x = self.layer4(x)
        x = self.bn4(x)
        x = self.act_fn(x)
        x = self.dropout2(x)
        
        x = self.layer5(x)
        x = self.bn5(x)
        x = self.act_fn(x)
        x = self.dropout2(x)
        
        x = self.layer6(x)
        x = self.bn6(x)
        x = self.act_fn(x)
        x = self.dropout2(x)
        
        x = self.layer7(x)
        x = self.bn7(x)
        x = self.act_fn(x)
        x = self.dropout2(x)
        
        x = self.layer8(x)
        x = self.bn8(x)
        x = self.act_fn(x)
        x = self.dropout2(x)

        x = self.out(x)
        
        return x

## Training

In [8]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [9]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [10]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 1000               # number of training epoch
learning_rate = 0.001       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-5)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
optimizer = radam.RAdam(model.parameters(), lr=learning_rate)
#lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 10, 0.2)

DEVICE: cuda


In [11]:
# start training

best_acc = 0.0
#best_loss = 1000.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
            #if val_loss < best_loss:
                best_acc = val_acc
                #best_loss = val_loss
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(val_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))
    #lr_scheduler.step()

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


/home/yiwei/ML/hw2/radam/radam.py:58: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)


[001/1000] Train Acc: 0.292743 Loss: 2.570507 | Val Acc: 0.447664 loss: 1.863715
saving model with acc 0.448
[002/1000] Train Acc: 0.509695 Loss: 1.633296 | Val Acc: 0.601607 loss: 1.308617
saving model with acc 0.602
[003/1000] Train Acc: 0.567423 Loss: 1.434306 | Val Acc: 0.644389 loss: 1.162333
saving model with acc 0.644
[004/1000] Train Acc: 0.599006 Loss: 1.322870 | Val Acc: 0.668748 loss: 1.063398
saving model with acc 0.669
[005/1000] Train Acc: 0.620674 Loss: 1.246921 | Val Acc: 0.684537 loss: 1.013119
saving model with acc 0.685
[006/1000] Train Acc: 0.634962 Loss: 1.193965 | Val Acc: 0.691627 loss: 0.985173
saving model with acc 0.692
[007/1000] Train Acc: 0.646188 Loss: 1.151516 | Val Acc: 0.703335 loss: 0.947691
saving model with acc 0.703
[008/1000] Train Acc: 0.656168 Loss: 1.115128 | Val Acc: 0.708018 loss: 0.922541
saving model with acc 0.708
[009/1000] Train Acc: 0.664085 Loss: 1.085693 | Val Acc: 0.713124 loss: 0.901532
saving model with acc 0.713
[010/1000] Train Ac

KeyboardInterrupt: 

In [16]:
optimizer = radam.RAdam(model.parameters(), lr=0.0001)
# start training

best_acc = 0.0
#best_loss = 1000.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
            #if val_loss < best_loss:
                best_acc = val_acc
                #best_loss = val_loss
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(val_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))
    #lr_scheduler.step()

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/1000] Train Acc: 0.761098 Loss: 0.723523 | Val Acc: 0.765745 loss: 0.720439
saving model with acc 0.766
[002/1000] Train Acc: 0.762390 Loss: 0.719281 | Val Acc: 0.766672 loss: 0.714885
saving model with acc 0.767
[003/1000] Train Acc: 0.764099 Loss: 0.713187 | Val Acc: 0.765159 loss: 0.725051
[004/1000] Train Acc: 0.764737 Loss: 0.710615 | Val Acc: 0.765972 loss: 0.716813
[005/1000] Train Acc: 0.765560 Loss: 0.708870 | Val Acc: 0.766867 loss: 0.712794
saving model with acc 0.767
[006/1000] Train Acc: 0.765800 Loss: 0.706724 | Val Acc: 0.766541 loss: 0.718818
[007/1000] Train Acc: 0.767049 Loss: 0.703501 | Val Acc: 0.766379 loss: 0.716772
[008/1000] Train Acc: 0.766999 Loss: 0.704303 | Val Acc: 0.766493 loss: 0.721991
[009/1000] Train Acc: 0.767343 Loss: 0.702299 | Val Acc: 0.766574 loss: 0.717801
[010/1000] Train Acc: 0.768166 Loss: 0.699730 | Val Acc: 0.767224 loss: 0.718137
saving model with acc 0.767
[011/1000] Train Acc: 0.767953 Loss: 0.699904 | Val Acc: 0.767615 loss: 0.7211

KeyboardInterrupt: 

In [21]:
optimizer = radam.RAdam(model.parameters(), lr=0.00005)

In [22]:
# start training

best_acc = 0.0
#best_loss = 1000.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
            #if val_loss < best_loss:
                best_acc = val_acc
                #best_loss = val_loss
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(val_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))
    #lr_scheduler.step()

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/1000] Train Acc: 0.768797 Loss: 0.697942 | Val Acc: 0.766883 loss: 0.719340
saving model with acc 0.767
[002/1000] Train Acc: 0.768425 Loss: 0.696686 | Val Acc: 0.766298 loss: 0.719098
[003/1000] Train Acc: 0.768390 Loss: 0.698017 | Val Acc: 0.766574 loss: 0.717034
[004/1000] Train Acc: 0.768899 Loss: 0.696996 | Val Acc: 0.766232 loss: 0.719945
[005/1000] Train Acc: 0.769029 Loss: 0.696411 | Val Acc: 0.767680 loss: 0.721514
saving model with acc 0.768
[006/1000] Train Acc: 0.769313 Loss: 0.696667 | Val Acc: 0.766785 loss: 0.718785
[007/1000] Train Acc: 0.769112 Loss: 0.694722 | Val Acc: 0.766574 loss: 0.721841
[008/1000] Train Acc: 0.769601 Loss: 0.694119 | Val Acc: 0.767241 loss: 0.718161
[009/1000] Train Acc: 0.769724 Loss: 0.694600 | Val Acc: 0.766737 loss: 0.721022
[010/1000] Train Acc: 0.769725 Loss: 0.693575 | Val Acc: 0.767387 loss: 0.716493


KeyboardInterrupt: 

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [23]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [24]:
predict = []
raw_output = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)
        for output in outputs.cpu().numpy():
            raw_output.append(output)

In [25]:
raw_output = np.array(raw_output)

In [ ]:
np.save("output", raw_output)

## Post-Processing

In [27]:
print('Loading data ...')

data_root='timit_11/timit_11/'
train_label = np.load(data_root + 'train_label_11.npy')

print('Size of training data: {}'.format(train_label.shape))

Loading data ...
Size of training data: (1229932,)


In [28]:
#transition matrix
trans_table = np.zeros((39,39))
train_label = train_label.astype('int')

for i in range(len(train_label)-1):
    trans_table[train_label[i], train_label[i+1]] += 1

In [29]:
trans_table_norm = trans_table / np.sum(trans_table,axis=1,keepdims=True)
trans_table_norm += 1e-17 #prevent dividing by zero
trans_table_norm = np.log(trans_table_norm)

In [31]:
m = nn.Softmax(dim = 1)
test_ln_softmax = m(torch.tensor(raw_output))
test_ln_softmax = np.array(test_ln_softmax)
test_ln_softmax = test_ln_softmax + 1e-17
test_ln_softmax = np.log(test_ln_softmax)

In [32]:
from tqdm import tqdm

In [33]:
tracking = np.zeros((451552, 39))
last_state = test_ln_softmax[0]
for i in tqdm(range(1,len(test_ln_softmax))):
    current_state = np.zeros(39)
    prob = last_state.reshape(39,1) + trans_table_norm + test_ln_softmax[i]
    current_state = np.max(prob, axis=0)
    tracking[i] = np.argmax(prob, axis=0)
    last_state = current_state

100%|██████████| 451551/451551 [00:08<00:00, 55648.90it/s]


In [35]:
np.argmax(raw_output[-1])

25

In [40]:
pred_ls = [25] #depend on last predict value

for i in range(0,451551):
    back = tracking[451552-i-1][int(pred_ls[-1])]
    pred_ls.append(int(back))
    
pred_ls = pred_ls[::-1]
predict = pred_ls

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

# Reference

1. [Sample Code](https://colab.research.google.com/github/ga642381/ML2021-Spring/blob/main/HW02/HW02-1.ipynb)
2. [Radam](https://github.com/LiyuanLucasLiu/RAdam/tree/master/radam)
3. [L. Li et al., "Hybrid Deep Neural Network--Hidden Markov Model (DNN-HMM) Based Speech Emotion Recognition," 2013 Humaine Association Conference on Affective Computing and Intelligent Interaction, Geneva, Switzerland, 2013, pp. 312-317, doi: 10.1109/ACII.2013.58.](https://ieeexplore.ieee.org/document/6681449)
4. [Abdel-rahman Mohamed, "Deep Neural Network acoustic models for ASR"](https://central.bac-lac.gc.ca/.item?id=TC-OTU-44123&op=pdf&app=Library&oclc_number=1032963679)

另外文章還有提到先用CD algorithm Pretrain 一個Deep Beliefs Network, 再利用這些weight訓練DNN, 這個方法礙於時間還沒嘗試, 應該可以做更好